## Review of Oedema (July 2025 and Oct 2025) 
#### Behavioural data modelling; splitting postop by site


**Notes:**\
When modelling the preliminary TN results (July 2025), noticed typo in BayesSMEP (where $\text{persev} \sim \mathcal{N}(\mu_{\text{persev}},\, \mu_{\text{persev}})$ instead of $\text{persev} \sim \mathcal{N}(\mu_{\text{persev}},\, \sigma_{\text{persev}})$. 

Fixing this led to almost no variability in per subject persev posteriors; after a bit of investigation; its down to the way distribution statements are constructed in stan https://mc-stan.org/docs/reference-manual/statements.html#distribution-statements.section, so the in-loop hyperparameter priors were raising the SD prior to the Nth power, as $p(\theta\mid y)\propto p(y\mid\theta)\,p(\theta)^N$ rather than as $p(\theta\mid y)\propto p(y\mid\theta)\,p(\theta)$ (NB: this prior statement and the original typo propagated directly from https://osf.io/vzs63/).

The corrections were justified with the generous help of the stan community https://discourse.mc-stan.org/t/hierarchical-model-behavioural-data-should-param-standard-deviation-priors-be-group-level-or-inside-per-subject-loop/40148/2 and availability of Jan Peters group's code https://osf.io/tvxgc/. As this coincided with the receipt of the Oedema paper review; a refit was required anyway as decided to split the post op behavioural data across sites.

In [ ]:
# preamble and imports 
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
import time 
from joblib import Parallel, delayed
import os

In [2]:
# Data: load in behavioural data
hc_df = pd.read_csv('../behavioural_data/Control_Thal/CompleteData.csv',header=0) 
pre_df = pd.read_csv('../behavioural_data/Pre_Thal/CompleteData.csv',header=0)
post_df = pd.read_csv('../behavioural_data/Post_Thal/CompleteData.csv',header=0)


hc_load = hc_df.to_numpy()
pre_load = pre_df.to_numpy()
post_load = post_df.to_numpy()

nSubjects_hc = 32
nTrials = int(hc_load.shape[0]/nSubjects_hc)

# reshaping for stan input
choice_hc = np.reshape(hc_load[:,3],(nSubjects_hc, nTrials))
reward_hc = np.reshape(hc_load[:,4],(nSubjects_hc, nTrials))

nSubjects_treat = 37

choice_pre = np.reshape(pre_load[:,3],(nSubjects_treat, nTrials))
reward_pre = np.reshape(pre_load[:,4],(nSubjects_treat, nTrials))

choice_post = np.reshape(post_load[:,3],(nSubjects_treat, nTrials))
reward_post = np.reshape(post_load[:,4],(nSubjects_treat, nTrials))

# indexing for Liverpool vs Dundee post thal subjects

liverpool_post_subjects = np.array([4,7,9,11,12,13,15,19,24,27,29,31])
dundee_post_subjects   = np.array([1,2,3,5,6,8,10,14,16,17,18,20,21,22,23,25,26,28,30,32,33,34,35,36,37])

liv_idx0 = liverpool_post_subjects - 1
dun_idx0 = dundee_post_subjects - 1

nSubjects_post_liv = len(liv_idx0) 
nSubjects_post_dun = len(dun_idx0)  

choice_post_liv  = choice_post[liv_idx0, :]
reward_post_liv  = reward_post[liv_idx0, :]

choice_post_dun  = choice_post[dun_idx0, :]
reward_post_dun  = reward_post[dun_idx0, :]
# print(choice_post_liv.shape)
# print(choice_post_dun.shape)
# print(reward_post_liv.shape)
# print(reward_post_dun.shape)    
# print(nSubjects_post_liv)
# print(nSubjects_post_dun)

In [3]:
models = ['AlphaSM', 'AlphaSME', 'AlphaSMP', 'AlphaSMEP','BayesSM', 'BayesSME', 'BayesSMP', 'BayesSMEP']
stan_files = [f'../Stan/stan_models/{prefix}_model.stan' for prefix in models]
executable_paths = []
model_list = []

for name, path in zip(models, stan_files):
    model = CmdStanModel(stan_file=path, force_compile=True)
    model_list.append(model)
    executable_paths.append(model.exe_file)

11:35:15 - cmdstanpy - INFO - compiling stan file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSM_model.stan to exe file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSM_model
11:35:27 - cmdstanpy - INFO - compiled model executable: /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSM_model
11:35:27 - cmdstanpy - INFO - compiling stan file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSME_model.stan to exe file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSME_model
11:35:40 - cmdstanpy - INFO - compiled model executable: /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSME_model
11:35:40 - cmdstanpy - INFO - compiling stan file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSMP_model.stan to exe file /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSMP_model
11:35:52 - cmdstanpy - INFO - compiled model executable: /home/isla/Projects/oedema_Oct25/Stan/stan_models/AlphaSMP_model
11:35:52 - cmdstanpy - INFO - compiling stan f

In [4]:
# data structures for stan
hc_data = {
    'nSubjects': nSubjects_hc,
    'nTrials': nTrials,
    'choice': choice_hc.astype(int),
    'reward': reward_hc
}

pre_treat_data = {
    'nSubjects': nSubjects_treat,
    'nTrials': nTrials,
    'choice': choice_pre.astype(int),
    'reward': reward_pre
}   

post_treat_data = {
    'nSubjects': nSubjects_treat,
    'nTrials': nTrials,
    'choice': choice_post.astype(int),
    'reward': reward_post
}

post_treat_liv_data = {
    'nSubjects': nSubjects_post_liv,
    'nTrials': nTrials,
    'choice': choice_post_liv.astype(int),
    'reward': reward_post_liv
}

post_treat_dun_data = {
    'nSubjects': nSubjects_post_dun,
    'nTrials': nTrials,
    'choice': choice_post_dun.astype(int),
    'reward': reward_post_dun
}

behavioural_data = [hc_data, pre_treat_data, post_treat_data, post_treat_liv_data, post_treat_dun_data]
groups = ['HC', 'PreTreat', 'PostTreat', 'PostTreat_Liv', 'PostTreat_Dun']

In [ ]:
# HMC 
out_root = 'Fits'
CHAINS= 4
nWarmup = 2500
nSamples = 2500

def fit_model_parallel(model_executable, data, outdir):
    print(f"Fitting model... {outdir}", flush=True)
    os.makedirs(outdir, exist_ok=True)

    model = CmdStanModel(exe_file=model_executable)
    fit = model.sample(data=data,chains=CHAINS,parallel_chains=CHAINS,iter_warmup=nWarmup,iter_sampling=nSamples,
       output_dir=outdir,show_console=False)

    print(f"Fitted {outdir}")
    for p in fit.runset.csv_files:
        print("  ", p)
    return fit

# context bound wrapper for the parallel jobs 
def run_fit(group_idx, model_idx):

    group = groups[group_idx]
    data             = behavioural_data[group_idx]
    model = models[model_idx]
    model_executable = executable_paths[model_idx]      

    outdir = os.path.join(out_root, group, model)
    fit = fit_model_parallel(model_executable, data, outdir)
    

In [ ]:
N_PARALLEL_JOBS = 5

# build a little job list

group_indices = range(len(groups))
model_indices = range(len(models))

jobs = [(group_idx, model_idx) for group_idx in group_indices for model_idx in model_indices ]

start = time.time()

results = Parallel(n_jobs=N_PARALLEL_JOBS, backend="loky", verbose=10)(
    delayed(run_fit)(group_idx, model_idx) for (group_idx, model_idx) in jobs
)
print(f"Elapsed: {time.time()- start:.3f} seconds")